<h1 style="text-align:center">Toronto Neighborhood Analysis - Capstone Project (IBM Data Science)</h1>

## Goal of this capstone project:

Use location data in order to compare Toronto's neighborhoods and cluster them into 5 distinct groups.
To do that, we will have to :

- Use location data and different location data providers, such as Foursquare.
- Make RESTful API calls to the Foursquare API to retrieve data about venues in different neighborhoods around the world.
- Be creative in situations where data are not readily available by scraping web data and parsing HTML code.
- Make use of Folium library to generate maps of geospatial data and to communicate results and findings.

## Table of content

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Retrieve the data from Wikipedia page](#retrieve_wikipedia)
2. [Data Wrangling](#data_wrangling)
3. [Exploration of the neighborhoods in Toronto](#exploration)
4. [Analysis of Each Neighborhood](#analysis_neighborhood)
5. [Cluster Neighborhoods](#cluster_neighborhoods)
6. [Examine Clusters](#clusters)

</div>
<hr>

Import all the libraries needed

In [1]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Retrieve the data from Wikipedia page <a id="retrieve_wikipedia"></a>

Let's first scrape our wikipedia page using __BeautifulSoup__.

In [2]:
import requests
postal_code_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(postal_code_url,'lxml')

Find class ‘wikitable sortable’ in the HTML script.

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})

Create the dataframe from html

In [5]:
df = pd.read_html(str(My_table))[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## 2. Data Wrangling <a id="data_wrangling"></a>

Drop the cells with a borough that is 'Not assigned'

In [6]:
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped rows
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<p>Here the Neighborhoods having the same Postal Code are already in the same raw, and separated with a coma.</p>
<p>Let's check that there are no missing values anymore :<p>

In [7]:
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Postal Code
False    103
Name: Postal Code, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighbourhood
False    103
Name: Neighbourhood, dtype: int64



OK, no missing values in our dataset !

In [8]:
print ("The size of the new dataset is: {} x {}".format(df.shape[0], df.shape[1]))

The size of the new dataset is: 103 x 3


In [9]:
# print a profile report of the dataset
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='report.html')

In order to retrieve the latitude and the longitude from the postal code, we will use the library __pgeocode__ :

In [10]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pgeocode

# we want to retrieve the latitude/longitude from a postal code in Canada 'ca'
nomi = pgeocode.Nominatim('ca')

latitude = []
longitude = []

for index, row in df.iterrows():
    location = nomi.query_postal_code(row[0])  # row[0] represents Postal Code value
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    
# we put the result of the loop in new columns 'latitude' and 'longitude'
df['Latitude'] = latitude
df['Longitude'] = longitude


# pb with Canada Post Gateway Processing Centre > need to do the query manually
df.loc[df['Neighbourhood'] == "Canada Post Gateway Processing Centre", ['Latitude', 'Longitude']] = [43.636966,-79.615819]

## 3. Exploration of the neighborhoods in Toronto <a id="exploration"></a>

### Create a map of Toronto with neighborhoods superimposed on top

Get the latitude and longitude of Toronto :

In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_long = location.longitude

In [18]:
# create map of TOronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto

### Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K' # your Foursquare ID
CLIENT_SECRET = 'CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K
CLIENT_SECRET:CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE


#### Let's explore the first neighborhood in our dataframe

In [20]:
# Get the neighborhood's latitude and longitude values
neighbourhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7545, -79.33.


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters

First, let's create the GET request URL :

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K&client_secret=CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE&v=20180605&ll=43.7545,-79.33&radius=500&limit=100'

Let's send the GET request and examine the resutls

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa9194b55636d4d71bc8526'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.7590000045, 'lng': -79.32378161085641},
   'sw': {'lat': 43.7499999955, 'lng': -79.33621838914358}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label'

 We know that all the information is in the *items* key. Before we proceed, let's create the **get_category_type** function :

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### Exploration of all the neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [27]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Let's check the size of the resulting dataframe

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(2172, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.7276,-79.3148,Tim Hortons,43.725517,-79.313103,Coffee Shop


## 4. Analysis of Each Neighborhood <a id="analysis_neighborhood"></a>

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,York Mills West,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2        Shanghai Restaurant   0.2
3            Badminton Court   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
               venue  freq
0        Pizza Place  0.14
1        Coffee Shop  0.14
2  Convenience Store  0.14
3           Pharmacy  0.14
4                Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0        Fried Chicken Joint  0.17
1                Pizza Place  0.17
2              Deli / Bodega  0.17
3  Middle Eastern Restaurant  0.17
4                Coffee Shop  0.17


----Bayview Village----
               venue  freq
0        Flower Shop  0.25
1              Trail  0.25
2        Gas Station  0.25
3               Park  0.25
4  Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant

                  venue  freq
0  Fast Food Restaurant  0.18
1         Grocery Store  0.09
2           Pizza Place  0.09
3                   Gym  0.09
4        Sandwich Place  0.09


----Stn A PO Boxes----
         venue  freq
0  Coffee Shop  0.16
1   Restaurant  0.07
2          Gym  0.04
3        Hotel  0.04
4         Café  0.03


----Studio District----
                   venue  freq
0  Performing Arts Venue  0.12
1          Garden Center  0.12
2                    Gym  0.12
3         Baseball Field  0.12
4                   Park  0.12


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                venue  freq
0         Coffee Shop  0.33
1  Light Rail Station  0.33
2        Liquor Store  0.17
3         Supermarket  0.17
4       Moving Target  0.00


----The Annex, North Midtown, Yorkville----
                venue  freq
0      Sandwich Place  0.10
1                Café  0.10
2  Italian Restaurant  0.05
3        Liquor Store  0.05
4         Pizza Place  0.05

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shanghai Restaurant,Latin American Restaurant,Badminton Court,Breakfast Spot,Skating Rink,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Gym,Coffee Shop,Convenience Store,Pharmacy,Cupcake Shop,Donut Shop,Fish & Chips Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Mediterranean Restaurant,Fried Chicken Joint,Deli / Bodega,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Health Food Store,Hawaiian Restaurant,Fish Market,Fish & Chips Shop
3,Bayview Village,Flower Shop,Park,Trail,Gas Station,Yoga Studio,Falafel Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Café,Liquor Store,Restaurant,Sushi Restaurant,Pub,Juice Bar


## 5. Cluster Neighborhoods <a id="cluster_neighborhoods"></a>

Run *k*-means to cluster the neighborhood into 5 clusters.

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with df (toronto data) to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

# Drop "na" values
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

# Cast Cluster Labels to int
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype("int64") 

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,3,Food & Drink Shop,Park,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
1,M4A,North York,Victoria Village,43.7276,-79.3148,0,Hockey Arena,Pizza Place,Coffee Shop,Intersection,Portuguese Restaurant,Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Yoga Studio,Beer Store,Restaurant,Event Space,Bakery,Thai Restaurant,Theater,Pub
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,0,Clothing Store,Coffee Shop,Women's Store,Cosmetics Shop,Restaurant,Jewelry Store,Shoe Store,Sushi Restaurant,Sandwich Place,Bakery
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Coffee Shop,Gym,Hobby Shop,Bubble Tea Shop,Martial Arts School,Café,Mexican Restaurant,Chinese Restaurant,Ethiopian Restaurant,Escape Room


Finally, let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters <a id="clusters"></a>

### Cluster 0

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Hockey Arena,Pizza Place,Coffee Shop,Intersection,Portuguese Restaurant,Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store
2,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Beer Store,Restaurant,Event Space,Bakery,Thai Restaurant,Theater,Pub
3,North York,0,Clothing Store,Coffee Shop,Women's Store,Cosmetics Shop,Restaurant,Jewelry Store,Shoe Store,Sushi Restaurant,Sandwich Place,Bakery
4,Downtown Toronto,0,Coffee Shop,Gym,Hobby Shop,Bubble Tea Shop,Martial Arts School,Café,Mexican Restaurant,Chinese Restaurant,Ethiopian Restaurant,Escape Room
5,Etobicoke,0,Pharmacy,Bank,Park,Grocery Store,Skating Rink,Falafel Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Salad Place,Steakhouse,Asian Restaurant,Seafood Restaurant
98,Etobicoke,0,Sushi Restaurant,Bank,Bakery,Breakfast Spot,Pub,Restaurant,Coffee Shop,Smoke Shop,Bar,Burger Joint
99,Downtown Toronto,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Bubble Tea Shop,Men's Store,Mediterranean Restaurant,Yoga Studio,Café
100,East Toronto,0,Restaurant,Coffee Shop,Japanese Restaurant,Martial Arts School,Breakfast Spot,Furniture / Home Store,Bookstore,Sushi Restaurant,Bank,Yoga Studio


### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Spa,Food Truck,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market


### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,2,Auto Garage,Yoga Studio,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field,Eastern European Restaurant


### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Food & Drink Shop,Park,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
6,Scarborough,3,Home Service,Yoga Studio,Fast Food Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field
7,North York,3,Park,Gym,Gym / Fitness Center,River,Pool,Trail,Yoga Studio,Eastern European Restaurant,Electronics Store,Escape Room
13,North York,3,Park,Gym,Gym / Fitness Center,River,Pool,Trail,Yoga Studio,Eastern European Restaurant,Electronics Store,Escape Room
21,York,3,Park,Women's Store,Beer Store,Mexican Restaurant,Bakery,Sporting Goods Shop,Gym,Yoga Studio,Farmers Market,Ethiopian Restaurant
27,North York,3,Residential Building (Apartment / Condo),Park,Yoga Studio,Farmers Market,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
32,Scarborough,3,Spa,Grocery Store,Park,Yoga Studio,Farmers Market,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
35,East York,3,Park,Convenience Store,Intersection,Fast Food Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
36,Downtown Toronto,3,Café,Harbor / Marina,Park,Music Venue,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio
39,North York,3,Flower Shop,Park,Trail,Gas Station,Yoga Studio,Falafel Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space


### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,North York,4,Bakery,Eastern European Restaurant,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
